In [3]:
import pandas as pd

# Read the Excel file
df = pd.read_excel('ONGB_EvalData_Complete_Anonymized.xlsx')

# Display basic info about the DataFrame
print(df.shape)
print(df.head())

(79460, 122)
   ANON_ID  Birthdate Gen          Eth_1718 Fluency_1718       SpEd_1718  \
0        1 1997-08-21   F             Asian         RFEP  Not Special Ed   
1        2 1999-10-10   F             Asian           EL  Not Special Ed   
2        3 2019-05-09   F               NaN          NaN             NaN   
3        4 2007-07-05   F  African American           EO  Not Special Ed   
4        5 2016-01-26   M               NaN          NaN             NaN   

              SiteName_1718 School Address_1718 City_1718  Zip_1718  ...  \
0  Oakland International HS     4521 Webster St   Oakland   94609.0  ...   
1  Oakland International HS     4521 Webster St   Oakland   94609.0  ...   
2                       NaN                 NaN       NaN       NaN  ...   
3         EnCompass Academy    1025 81st Avenue   Oakland   94621.0  ...   
4                       NaN                 NaN       NaN       NaN  ...   

   Grade_2324  AttRate_2324  DaysEnr_2324  DaysAbs_2324  Susp_2324  \
0  

In [4]:
import re
# Identify columns with year suffixes (1718, 2324, etc.)
year_pattern = r'_(\d{4})$'
year_cols = [col for col in df.columns if re.search(year_pattern, col)]

# ID columns are those WITHOUT year suffixes
id_cols = [col for col in df.columns if col not in year_cols]

# Pivot from wide to long format
df_long = df.melt(
    id_vars=id_cols,
    var_name='Variable',
    value_name='Value'
)

# Extract school year and variable name
df_long[['VarName', 'SchoolYear']] = df_long['Variable'].str.extract(
    r'(.+?)_(\d{4})$'
)

# Clean up
df_long = df_long.drop('Variable', axis=1)

# Reorder columns for clarity
df_long = df_long[['ANON_ID', 'SchoolYear', 'VarName', 'Value'] + 
                   [col for col in id_cols if col != 'ANON_ID']]

print(f"Pivoted shape: {df_long.shape}")
print(df_long.head(20))

# Optional: Save the pivoted data
df_long.to_csv('ONGB_EvalData_Pivoted.csv', index=False)
print("\nPivoted data saved to ONGB_EvalData_Pivoted.csv")

Pivoted shape: (8343300, 20)
    ANON_ID SchoolYear VarName               Value  Birthdate Gen City_1718.1  \
0         1       1718     Eth               Asian 1997-08-21   F     Oakland   
1         2       1718     Eth               Asian 1999-10-10   F     Oakland   
2         3       1718     Eth                 NaN 2019-05-09   F         NaN   
3         4       1718     Eth    African American 2007-07-05   F     Oakland   
4         5       1718     Eth                 NaN 2016-01-26   M         NaN   
5         6       1718     Eth                 NaN 2018-11-16   M         NaN   
6         7       1718     Eth              Latino 2003-06-09   F     Oakland   
7         8       1718     Eth              Latino 2012-05-19   F     Oakland   
8         9       1718     Eth              Latino 2006-11-16   M     Oakland   
9        10       1718     Eth              Latino 2011-05-14   F     Oakland   
10       11       1718     Eth                 NaN 2017-02-22   M         NaN   

In [5]:
# Example: Convert numeric variables to float
numeric_vars = ['Grade', 'AttRate', 'DaysEnr', 'DaysAbs', 'GPA']
df_long.loc[df_long['VarName'].isin(numeric_vars), 'Value'] = \
    pd.to_numeric(df_long.loc[df_long['VarName'].isin(numeric_vars), 'Value'], errors='coerce')

In [6]:
df_long = df_long.dropna(subset=['Value'])

In [8]:
# All attendance rates
att_data = df_long[df_long['VarName'] == 'AttRate']

# One student across all years
student_1 = df_long[df_long['ANON_ID'] == 1]

In [9]:
print("=" * 80)
print("STEP 1: UNDERSTANDING YOUR CURRENT DATA")
print("=" * 80)

print(f"\nShape: {df_long.shape}")
print(f"  Total rows: {df_long.shape[0]:,}")
print(f"  Total columns: {df_long.shape[1]}")

print(f"\n\nColumn names:")
print(df_long.columns.tolist())

print(f"\n\nFirst 10 rows:")
print(df_long.head(10))

print(f"\n\nData types:")
print(df_long.dtypes)

print(f"\n\nMissing values overview:")
print(df_long.isnull().sum())

STEP 1: UNDERSTANDING YOUR CURRENT DATA

Shape: (3653408, 20)
  Total rows: 3,653,408
  Total columns: 20


Column names:
['ANON_ID', 'SchoolYear', 'VarName', 'Value', 'Birthdate', 'Gen', 'City_1718.1', 'Zip_1718.1', 'City_1819.1', 'Zip_1819.1', 'City_1920.1', 'Zip_1920.1', 'City_2021.1', 'Zip_2021.1', 'City_2122.1', 'Zip_2122.1', 'City_2223.1', 'Zip_2223.1', 'City_2324.1', 'Zip_2324.1']


First 10 rows:
    ANON_ID SchoolYear VarName               Value  Birthdate Gen City_1718.1  \
0         1       1718     Eth               Asian 1997-08-21   F     Oakland   
1         2       1718     Eth               Asian 1999-10-10   F     Oakland   
3         4       1718     Eth    African American 2007-07-05   F     Oakland   
6         7       1718     Eth              Latino 2003-06-09   F     Oakland   
7         8       1718     Eth              Latino 2012-05-19   F     Oakland   
8         9       1718     Eth              Latino 2006-11-16   M     Oakland   
9        10       1718   

In [10]:
# STEP 2: Keep only constant demographic columns (one per student)
print("\n" + "=" * 80)
print("STEP 2: REMOVING DUPLICATE DEMOGRAPHIC INFO")
print("=" * 80)

print(f"\nBefore: {df_long.shape}")

# Keep only essential columns - drop the year-specific City/Zip columns
# We'll keep Birthdate and Gen since they're student-level constants
df_long = df_long[['ANON_ID', 'SchoolYear', 'VarName', 'Value', 'Birthdate', 'Gen']]

print(f"After dropping year-specific location columns: {df_long.shape}")

print(f"\nNew columns: {df_long.columns.tolist()}")
print(f"\nFirst 10 rows:")
print(df_long.head(10))


STEP 2: REMOVING DUPLICATE DEMOGRAPHIC INFO

Before: (3653408, 20)
After dropping year-specific location columns: (3653408, 6)

New columns: ['ANON_ID', 'SchoolYear', 'VarName', 'Value', 'Birthdate', 'Gen']

First 10 rows:
    ANON_ID SchoolYear VarName               Value  Birthdate Gen
0         1       1718     Eth               Asian 1997-08-21   F
1         2       1718     Eth               Asian 1999-10-10   F
3         4       1718     Eth    African American 2007-07-05   F
6         7       1718     Eth              Latino 2003-06-09   F
7         8       1718     Eth              Latino 2012-05-19   F
8         9       1718     Eth              Latino 2006-11-16   M
9        10       1718     Eth              Latino 2011-05-14   F
12       13       1718     Eth            Filipino 2011-08-28   F
14       15       1718     Eth  Multiple Ethnicity 1999-12-06   M
15       16       1718     Eth    African American 2000-05-25   M


In [11]:
# STEP 3: CONVERT VALUE COLUMN TO PROPER DATA TYPES
print("\n" + "=" * 80)
print("STEP 3: CONVERTING VALUE TO NUMERIC WHERE APPROPRIATE")
print("=" * 80)

# First, let's see what variables you have
print(f"\nUnique variables in your data:")
print(df_long['VarName'].unique())
print(f"\nTotal unique variables: {df_long['VarName'].nunique()}")

# Show sample values for each variable
print(f"\n\nSample values by variable:")
for var in df_long['VarName'].unique()[:10]:
    samples = df_long[df_long['VarName'] == var]['Value'].unique()[:3]
    print(f"  {var}: {samples}")


STEP 3: CONVERTING VALUE TO NUMERIC WHERE APPROPRIATE

Unique variables in your data:
['Eth' 'Fluency' 'SpEd' 'SiteName' 'School Address' 'City' 'Zip' 'Grade'
 'AttRate' 'DaysEnr' 'DaysAbs' 'Susp' 'Address' 'CurrWeightedTotGPA' 'SED']

Total unique variables: 15


Sample values by variable:
  Eth: ['Asian' 'African American' 'Latino']
  Fluency: ['RFEP' 'EL' 'EO']
  SpEd: ['Not Special Ed' 'Special Ed']
  SiteName: ['Oakland International HS' 'EnCompass Academy' 'MetWest High School']
  School Address: ['4521 Webster St' '1025 81st Avenue' '314 E 10th St']
  City: ['Oakland']
  Zip: [94609.0 94621.0 94606.0]
  Grade: [12.0 5.0 9.0]
  AttRate: [0.9722 0.9278 0.9943]
  DaysEnr: [180.0 174.0 119.0]


In [12]:
# STEP 3 (continued): Actually convert numeric variables
print("\n" + "=" * 80)
print("STEP 3: CONVERTING NUMERIC VARIABLES")
print("=" * 80)

# Define which variables are numeric
numeric_vars = ['Grade', 'AttRate', 'DaysEnr', 'DaysAbs', 'Susp', 'CurrWeightedTotGPA']

# For each numeric variable, convert its values to float
for var in numeric_vars:
    mask = df_long['VarName'] == var
    df_long.loc[mask, 'Value'] = pd.to_numeric(
        df_long.loc[mask, 'Value'], 
        errors='coerce'  # converts non-numeric to NaN
    )
    print(f"  ✓ Converted {var} to numeric")

print(f"\n\nValue column data type after conversion:")
print(df_long['Value'].dtype)

print(f"\n\nSample of converted data:")
print(df_long[df_long['VarName'] == 'AttRate'][['ANON_ID', 'SchoolYear', 'VarName', 'Value']].head(10))

print(f"\n\nSample of categorical data (unchanged):")
print(df_long[df_long['VarName'] == 'Eth'][['ANON_ID', 'SchoolYear', 'VarName', 'Value']].head(10))


STEP 3: CONVERTING NUMERIC VARIABLES
  ✓ Converted Grade to numeric
  ✓ Converted AttRate to numeric
  ✓ Converted DaysEnr to numeric
  ✓ Converted DaysAbs to numeric
  ✓ Converted Susp to numeric
  ✓ Converted CurrWeightedTotGPA to numeric


Value column data type after conversion:
object


Sample of converted data:
        ANON_ID SchoolYear  VarName   Value
635680        1       1718  AttRate  0.9722
635681        2       1718  AttRate  0.9278
635683        4       1718  AttRate  0.9943
635687        8       1718  AttRate  0.9778
635688        9       1718  AttRate  0.9889
635689       10       1718  AttRate  0.9944
635692       13       1718  AttRate  0.9833
635694       15       1718  AttRate  0.9222
635695       16       1718  AttRate  0.9664
635698       19       1718  AttRate  0.9389


Sample of categorical data (unchanged):
    ANON_ID SchoolYear VarName               Value
0         1       1718     Eth               Asian
1         2       1718     Eth               Asian
3

In [14]:
# STEP 4: SAVE YOUR CLEANED DATA (CSV only)
print("\n" + "=" * 80)
print("STEP 4: SAVING CLEANED DATA")
print("=" * 80)

print(f"\nFinal cleaned dataset:")
print(f"  Shape: {df_long.shape}")
print(f"  Rows: {df_long.shape[0]:,}")
print(f"  Columns: {df_long.columns.tolist()}")

print(f"\n\nData summary:")
print(f"  Students: {df_long['ANON_ID'].nunique():,}")
print(f"  School years: {sorted(df_long['SchoolYear'].unique())}")
print(f"  Variables tracked: {df_long['VarName'].nunique()}")

print(f"\n\nNull values in Value column by variable:")
null_summary = df_long.groupby('VarName')['Value'].apply(lambda x: x.isna().sum())
print(null_summary)

# Save the cleaned data as CSV
df_long.to_csv('ONGB_EvalData_Cleaned_Pivoted.csv', index=False)
print(f"\n\n✓ CLEANED DATA SAVED: ONGB_EvalData_Cleaned_Pivoted.csv")

print(f"\n{'='*80}")
print("✓ YOU'RE DONE WITH CLEANING!")
print(f"{'='*80}")
print(f"\nYour data is now ready for analysis:")
print(f"  - Pivoted from wide to long format")
print(f"  - Removed duplicate demographic columns")
print(f"  - Converted numeric variables to proper types")
print(f"  - Saved and ready to explore!")


STEP 4: SAVING CLEANED DATA

Final cleaned dataset:
  Shape: (3653408, 6)
  Rows: 3,653,408
  Columns: ['ANON_ID', 'SchoolYear', 'VarName', 'Value', 'Birthdate', 'Gen']


Data summary:
  Students: 79,460
  School years: ['1718', '1819', '1920', '2021', '2122', '2223', '2324']
  Variables tracked: 15


Null values in Value column by variable:
VarName
Address               0
AttRate               0
City                  0
CurrWeightedTotGPA    0
DaysAbs               0
DaysEnr               0
Eth                   0
Fluency               0
Grade                 0
SED                   0
School Address        0
SiteName              0
SpEd                  0
Susp                  0
Zip                   0
Name: Value, dtype: int64


✓ CLEANED DATA SAVED: ONGB_EvalData_Cleaned_Pivoted.csv

✓ YOU'RE DONE WITH CLEANING!

Your data is now ready for analysis:
  - Pivoted from wide to long format
  - Removed duplicate demographic columns
  - Converted numeric variables to proper types
  - Save

# ONGB Data Cleaning Pipeline - Bullet Point Summary

## Starting Point
- **Raw data**: 79,460 students × 122 columns (WIDE FORMAT)
- **Problem**: Each school year had its own columns (Eth_1718, Eth_1819, Grade_1718, Grade_1819, etc.)
- **Issue**: Hard to analyze trends across time; data scattered across many columns

---

## STEP 1: PIVOT FROM WIDE TO LONG FORMAT
**What we did**: Used `df.melt()` to transform data structure
- Identified columns with year suffixes (1718, 1819, etc.)
- Combined them into 3 key columns: SchoolYear, VarName, Value
- Extracted school year and variable names using regex

**Result**: 
- 79,460 rows → 3,653,408 rows (one row per student-year-variable combo)
- 122 columns → 20 columns initially

**Why**: 
- Makes time-series analysis possible
- Each observation is now one row (tidy data principle)
- Easy to filter: `df[df['VarName'] == 'AttRate']` gets all attendance data
- Easy to visualize trends across years

---

## STEP 2: REMOVE DUPLICATE DEMOGRAPHIC COLUMNS
**What we did**: Dropped year-specific City/Zip columns
- Removed: City_1718, Zip_1718, City_1819, Zip_1819, etc. (10 redundant columns)
- Kept: ANON_ID, SchoolYear, VarName, Value, Birthdate, Gen

**Result**:
- 20 columns → 6 columns (70% reduction)
- 3,653,408 rows (unchanged)

**Why**:
- Birthdate and Gen don't change year-to-year; no need to repeat them
- City/Zip change by year but are incomplete (lots of NaN)
- Location is student metadata, not an observation metric
- Smaller file = faster to work with
- Cleaner, more focused dataset for analysis

---

## STEP 3: CONVERT VALUE COLUMN TO PROPER DATA TYPES
**What we did**: Converted numeric variables from text to float
- Identified 6 numeric variables: Grade, AttRate, DaysEnr, DaysAbs, Susp, CurrWeightedTotGPA
- Used `pd.to_numeric()` to convert text → float
- Left 9 categorical variables as text: Eth, Fluency, SpEd, SiteName, etc.

**Result**:
- Numeric variables can now be used in calculations
- Categorical variables remain text for grouping/filtering

**Why**:
- Without conversion, you can't calculate mean, compare values, or do math
- `df['Value'].mean()` would fail if Value is all text
- `df['Value'] > 0.90` would fail for text data
- Now you can: `attendance.mean()`, `attendance.std()`, `attendance > 0.90`, etc.

---

## STEP 4: SAVE CLEANED DATA
**What we did**: Exported to CSV
- File: `ONGB_EvalData_Cleaned_Pivoted.csv`

**Why CSV**:
- Universal format (works in Excel, R, Python, SQL)
- Good balance of size and readability
- Can handle 3.6+ million rows
- No special software required

---

## FINAL DATASET

**Shape**: 3,653,408 rows × 6 columns

**Columns**:
- `ANON_ID` - Student identifier
- `SchoolYear` - School year (1718, 1819, 1920, 2021, 2122, 2223, 2324)
- `VarName` - Variable name (15 different variables)
- `Value` - Actual data (numeric or categorical)
- `Birthdate` - Student birth date
- `Gen` - Student gender (F/M)

**Data Scope**:
- 79,460 unique students
- 7 school years (2017-2024)
- 15 variables tracked
- 0 null values in Value column

---

## KEY TRANSFORMATIONS AT A GLANCE

| Before | After |
|--------|-------|
| Wide format (columns for each year) | Long format (one row per observation) |
| 122 columns | 6 columns |
| Hard to analyze trends | Easy to analyze trends |
| Redundant demographic data | Focused dataset |
| Mixed data types | Proper data types |
| Ready for entry, not analysis | Ready for analysis |

---

## NOW YOU CAN...

✅ Track individual students across years  
✅ Calculate average attendance by school year  
✅ Find students with chronic absenteeism (AttRate < 0.90)  
✅ Compare demographics with academic outcomes  
✅ Visualize trends across time  
✅ Build predictive models  
✅ Do statistical analysis  

---

## QUICK REFERENCE: VARIABLE TYPES

**Numeric** (can do math on these):
- Grade, AttRate, DaysEnr, DaysAbs, Susp, CurrWeightedTotGPA

**Categorical** (use for grouping/filtering):
- Eth, Fluency, SpEd, SiteName, School Address, City, Zip, Address, SED